In [1]:
import pandas as pd

def extractDfFromFile(nFiles, filePrefix, headerFlag):
    df = pd.DataFrame()
    for i in range(1,nFiles+1): # loops from 1 to nFiles
        if headerFlag == 0:
            entries = pd.read_csv(filePrefix+str(i)+'.txt', sep = '@', header = None)
        else:
            entries = pd.read_csv(filePrefix+str(i)+'.txt', sep = '@')
        df = df.append(pd.DataFrame(entries))
    return df

In [2]:
# tip data: Build user id, compliment_count df from extracted tip data 
filePath = '/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/'
filePrefix = filePath+'tips_'
tipsDf = extractDfFromFile(13, filePrefix, 1)

#aggregate compliment count
tipsDf = tipsDf.groupby(['user_id']).sum()
print('end')

end


In [3]:
# user data: Build user id, user name df from extracted user data 
filePath = '/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/'
filePrefix = filePath+'userids_'
usersDf = extractDfFromFile(17, filePrefix, 0)

usersDf = usersDf.rename(columns={0: "user_id", 1: "user_name"})
#print(usersDf.head())
print('end')

end


In [4]:
#Merge user and tip and dataframes
user_tipDf = pd.merge(usersDf, tipsDf, on='user_id', how = 'inner', sort = False, validate = 'one_to_one' )
print(user_tipDf.head())

                  user_id user_name  compliment_count
0  4XChL029mKr5hydo79Ljxg     Jenna                 0
1  bc8C_eETBWL0olvFSJJd0w     David                 0
2  MM4RJAeH6yuaN8oZDSt0RA     Nancy                 0
3  0rK89TS8xqy1wI4nYI1wfw   Marilyn                 0
4  T0gWkTHWRChVUe_Dn1F8nw     Tanya                 0


In [5]:
# review data: Build review id, userid, businessid df from extracted review data 
filePath = '/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/review/'
filePrefix = filePath+'reviews_'
reviewsDf = extractDfFromFile(67, filePrefix, 1)
print('end')

end


In [6]:
#business data: business id, categories
filePath = '/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/'
filePrefix = filePath+'business_'
businessDf = extractDfFromFile(2, filePrefix, 1)
#print(businessDf.head())
print('end')

end


In [7]:
import numpy as np

categories = businessDf['categories']
catSet = set()
for cat in categories:
    catSet.update(set(cat.split(', ')))
print(len(catSet))

def countCats(categoriesRow):
    currCat = categoriesRow['category']
    business_matching_currCategory = businessDf[ businessDf['categories'].str.contains(currCat+",")]
    busCount = business_matching_currCategory['business_id'].unique().size
    return busCount

categoriesDF = pd.DataFrame(columns = ['category', 'businessCount'])
categoriesDF['category'] = list(catSet)
categoriesDF['businessCount'] = np.zeros(len(categoriesDF['category']))
categoriesDF['businessCount'] = categoriesDF.apply(lambda x:  countCats(x), axis = 1)
print('end')

1301


/Users/vcroopana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # This is added back by InteractiveShellApp.init_path()


end


In [8]:
# categoriesDF.sort_values(by = 'businessCount', ascending=False, inplace=True)
# print(categoriesDF.head(10))
# outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
# categoriesDF.to_csv(outputFilePath+'categories'+".csv", index=False)


In [9]:
#Create dic of updated Categories
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
categoriesDFUpdated = pd.read_csv(outputFilePath+'categories_updated'+".csv")

categoriesDict = {}
nCatsOriginal = categoriesDFUpdated['category'].size
for i in range(0, nCatsOriginal):
    categoriesDict[categoriesDFUpdated.iloc[i]['category']] = categoriesDFUpdated.iloc[i]['newCategory']

In [10]:
#create all category cols in business df
uniqueNewCategories = categoriesDFUpdated['newCategory'].unique()
for newCat in uniqueNewCategories:
    businessDf[newCat] = False
print('end')

end


In [11]:
# adds new column 'newCategories' in business dataframe
import timeit
import pandas as pd

def updateEachCatDict(business):
    currCats = business['categories'].split(', ')
    updatedCategory = ""
    i = 0
    for cat in currCats:
        updatedSingleCategory = categoriesDict[cat]
        business[updatedSingleCategory] = True
        if i == 0:
            updatedCategory = categoriesDict[cat]
        else:
            value = categoriesDict[cat]
            if value.casefold() not in updatedCategory.casefold():
                updatedCategory = updatedCategory + ", " + value
        i = i+1
    business['newCategories'] = updatedCategory
    return business

start_time = timeit.default_timer()

businessDf['newCategories'] = ''
businessDf = businessDf.apply(lambda x:  updateEachCatDict(x), axis = 1)

elapsed = timeit.default_timer() - start_time
print(elapsed)

307.789729781


In [12]:
review_businessDf = pd.merge(reviewsDf, businessDf, on='business_id', how = 'left', sort = False)
print('end')

end


In [ ]:
uniqueNewCategories = categoriesDFUpdated['newCategory'].unique()

user_cat_df = pd.DataFrame()
user_cat_df['user_id'] = review_businessDf['user_id'].unique()
print('Total no of users:'+ str(user_cat_df['user_id'].size))

for cat in uniqueNewCategories:
    
    userGroups = review_businessDf[review_businessDf[cat] == True].groupby(['user_id'])
    print(userGroups)
    catUserRatings = userGroups['rating'].mean()
    print('Size of users in category, '+ cat + ":" + str(catUserRatings.size))
    catUserRatings = pd.DataFrame({'user_id':catUserRatings.index, cat: catUserRatings.values})
    user_cat_df = pd.merge(user_cat_df, catUserRatings, on ='user_id', how = 'left', sort = False )

print(user_cat_df)

In [14]:
print(user_cat_df.shape)
user_cat_df_0 = user_cat_df.fillna(0)
user_cat_0_matrix = user_cat_df_0.drop(['user_id'], axis = 1).values
print(user_cat_0_matrix)

(1637138, 34)
[[2.2        2.         1.         ... 1.         0.         2.33333333]
 [3.         0.         0.         ... 0.         0.         3.        ]
 [5.         5.         0.         ... 0.         0.         0.        ]
 ...
 [5.         0.         0.         ... 0.         0.         0.        ]
 [5.         0.         0.         ... 0.         0.         5.        ]
 [5.         0.         0.         ... 0.         0.         0.        ]]


In [15]:
#user_user = user_cat_0_matrix.dot(user_cat_0_matrix.transpose())
print('end')
#print(user_user)
print(user_cat_df.shape)

end
(1637138, 34)


In [17]:
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
user_cat_df_0.to_csv(outputFilePath + 'user_category'+".csv", index=False)

In [18]:
print(user_cat_df_0.isin([0]).sum())

0


In [ ]:
# from scipy.sparse import csr_matrix

# sparseMatrix = csr_matrix(user_cat_0_matrix)
# user_user_sparse = sparseMatrix.dot(csr_matrix(user_cat_0_matrix.transpose()))
# print(user_user_sparse)

In [2]:
import pandas as pd
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
df = pd.read_csv(outputFilePath+'user_category'+".csv")
print('end')

end


In [5]:
import sklearn.preprocessing as pp
from scipy.sparse import csr_matrix, csc_matrix
import pandas as pd
import numpy as np
import scipy.linalg.blas

def cosine_similarities(mat):

    row_normed_mat = pp.normalize(csr_matrix(mat), axis=1) # 1 => normalizes each row, doing this for cosine similarity
    #nRows = row_normed_mat.shape[0]
    nRows = 1
    count = 1
    filecount = 1
    outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
    outputFilePrefix = 'user_user_'
    writefile= open(outputFilePath+outputFilePrefix+"1.txt","w+")
    header = 'user_index@most_similar_user_index'
    writefile.writelines(header)
    maxInd =0
    for i in range(0, nRows):
        #val = row_normed_mat.getrow(i) * row_normed_mat.T
        
        val = scipy.linalg.blas.sgemm(alpha=1.0, a= row_normed_mat.getrow(i).toarray(), b= row_normed_mat.toarray(), trans_b=True)
        #maxInd = np.argsort(val.getrow(0).toarray().flatten())[-2]
        maxInd = np.argsort(val.flatten())[-2]
        if count == (filecount * 100000): # change to new file for every 1 lakh rows
            writefile.close()
            filecount = filecount+1
            print("count: "+ str(count) + " curr filecount: "+ str(filecount))
            writefile= open(outputFilePath+outputFilePrefix+str(filecount)+".txt","w+")       
            writefile.writelines(header)
        
        writefile.writelines(str(i)+ "@" + str(maxInd))
        count = count+1
        print(count)
    writefile.close()
    return maxInd
    #return row_normed_mat * row_normed_mat.T

print('started')
vals = df.drop(['user_id'], axis = 1).to_numpy(dtype= 'float32')
cosineSims = cosine_similarities(vals)

print(cosineSims)

# use triangular matrix 
# or use 1 by 16 lakhs
print('end')

started
2
300256
end


In [66]:
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

vals = df.drop(['user_id'], axis = 1).to_numpy(dtype= 'float32')
nRows = 1637138
blockSize = 1000
nBlocks = 1630


filecount = 1
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
outputFilePrefix = 'user_user_'
writefile= open(outputFilePath+outputFilePrefix+"1.txt","w+")
header = 'user_index@most_similar_user_index@value\n'
writefile.writelines(header)

for i in range(999, nRows):
    maxInd =0
    maxVal =0
    count = i + 1
    for j in range(0 , nBlocks):
        if j == 0:
            blockStartInd = j*blockSize
        else:
            blockStartInd = (j*blockSize) + 1
        
        blockEndInd = (j+1)*blockSize
        blockShape = csr_matrix(vals[blockStartInd:blockEndInd,:]).shape
        if(blockShape[0]!=0):
            res = cosine_similarity(csr_matrix(vals[i]), csr_matrix(vals[blockStartInd:blockEndInd ,:]))
            actualInd = (rowStartInd + i)
                
            if actualInd in range(blockStartInd,blockEndInd+1):
                res[0, i] = 0
                
            #tempMaxInd = np.maximum(res.flatten())[-2] #tempMaxVal = res.flatten()[tempMaxInd] #tempMaxVal = res.max()
            tempMaxInd = np.argmax(res[0])
            tempMaxVal = res[0, tempMaxInd]
            tempMaxInd = blockStartInd - 1 + tempMaxInd
            if(tempMaxVal> maxVal):
                maxInd = tempMaxInd
                maxVal = tempMaxVal
    if count == (filecount * 100000): # change to new file for every 1 lakh rows
        writefile.close()
        filecount = filecount+1
        print("count: "+ str(count) + " curr filecount: "+ str(filecount))
        writefile= open(outputFilePath+outputFilePrefix+str(filecount)+".txt","w+")       
        writefile.writelines(header)
    writefile.writelines(str(i)+"@"+str(maxInd)+"@"+str(maxVal)+"\n")
    print(str(i)+"@"+str(maxInd)+"@"+str(maxVal))
    if(i%100 ==0):
        print(str(i)+" maxInd:"+ str(maxInd)+"@"+str(maxVal))

writefile.close()

NameError: name 'rowStartInd' is not defined

In [ ]:
# import tables
# import time

# h5f_C.close()
# A = user_cat_0_matrix
# B = user_cat_0_matrix.transpose()

# fileName_C = outputFilePath+ 'CArray_C.h5'
# atom = tables.Float32Atom()
# nUsers = user_cat_df_0.shape[0]
# shape = (nUsers, nUsers)
# nChunk = 500 
# chunk_shape = (nChunk, nChunk)
# chunk_multiple = 1 #?
# block_size = chunk_multiple * nChunk
# h5f_C = tables.open_file(fileName_C, 'w')
# C = h5f_C.create_carray(h5f_C.root, 'CArray', atom, shape, chunkshape=chunk_shape)

# sz= block_size

# t0= time.time()
# for i in range(0, A.shape[0], sz):
#     for j in range(0, B.shape[1], sz):
#         for k in range(0, A.shape[1], sz):
#             C[i:i+sz,j:j+sz] += np.dot(A[i:i+sz,k:k+sz],B[k:k+sz,j:j+sz])
# print (time.time()-t0)

# t0 = time.time()
# res = np.dot(A,B)
# print (time.time()-t0)

# print (C == res)

# h5f_C.close()